<a href="https://colab.research.google.com/github/alkhalisy/yolov8-gam/blob/main/yolov8_gam_test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [2]:
%cd {HOME}
!git clone https://github.com/alkhalisy/yolov8-gam.git
%cd /content/yolov8-gam
!pip install -e .


/content
Cloning into 'yolov8-gam'...
remote: Enumerating objects: 295, done.
remote: Counting objects: 100% (295/295), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 295 (delta 76), reused 249 (delta 52), pack-reused 0
Receiving objects: 100% (295/295), 738.40 KiB | 17.58 MiB/s, done.
Resolving deltas: 100% (76/76), done.
/content/yolov8-gam
Obtaining file:///content/yolov8-gam
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.

In [3]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.2/78.2 GB disk)


In [4]:
from ultralytics import YOLO

In [14]:
!yolo detect train data=coco8.yaml model = yolov8s-gam.yaml pretrained=yolov8s.pt epochs=100 imgsz=640

Transferred 150/373 items from pretrained weights
Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s-gam.yaml, data=coco8.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=yolov8s.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cr

In [9]:
# Download COCO val
import torch
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d datasets && rm tmp.zip  # unzip

100%|██████████| 780M/780M [00:04<00:00, 178MB/s]


In [10]:
!yolo val model="/content/YOLOv8-SE/runs/detect/train/weights/best.pt" data=coco8.yaml

Ultralytics YOLOv8.2.45 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8_test3 summary (fused): 179 layers, 3620128 parameters, 0 gradients, 9.2 GFLOPs
val: Scanning /content/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100% 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 1/1 [00:00<00:00,  3.31it/s]
                   all          4         17          0          0          0          0
Speed: 0.3ms preprocess, 51.2ms inference, 0.0ms loss, 8.4ms postprocess per image
Results saved to /content/YOLOv8-SE/runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


In [10]:
from IPython.display import display, Image

In [16]:
!mkdir {HOME}/datasets2
%cd {HOME}/datasets2

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="voj48YaWzt61e5xEk3jK")
project = rf.workspace("test-e703r").project("ai2sd_test")
dataset = project.version(1).download("yolov8")


/content/YOLOv8-SE/datasets2
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.45, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to ai2sd_test-1 in yolov8:: 100%|██████████| 5581/5581 [00:01<00:00, 5013.69it/s]


In [18]:
import yaml

# Define the file path
file_path = "/content/YOLOv8-SE/datasets2/ai2sd_test-1/data.yaml"

# Read the YAML file
with open(file_path, "r") as file:
    data = yaml.safe_load(file)

# Make the desired modifications
data['train'] = '/content/YOLOv8-SE/datasets2/ai2sd_test-1/train/images'
data['val'] = '/content/YOLOv8-SE/datasets2/ai2sd_test-1/valid/images'

# Write the modified data back to the file
with open(file_path, "w") as file:
    yaml.dump(data, file, default_flow_style=False)


In [19]:
!yolo detect val model="/content/YOLOv8-SE/runs/detect/train2/weights/best.pt" data="/content/YOLOv8-SE/datasets2/ai2sd_test-1/data.yaml"

Ultralytics YOLOv8.2.45 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8_test2 summary (fused): 174 layers, 3193136 parameters, 0 gradients, 8.9 GFLOPs
val: Scanning /content/YOLOv8-SE/datasets2/ai2sd_test-1/valid/labels... 101 images, 0 backgrounds, 0 corrupt: 100% 101/101 [00:00<00:00, 1960.81it/s]
val: New cache created: /content/YOLOv8-SE/datasets2/ai2sd_test-1/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 7/7 [00:04<00:00,  1.69it/s]
                   all        101        123          0          0          0          0
Speed: 7.9ms preprocess, 15.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/YOLOv8-SE/runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val
